In [34]:
# from ipyleaflet import Map, Marker, Icon, CircleMarker, MarkerCluster, AwesomeIcon, Popup, basemaps, basemap_to_tiles
# from ipywidgets import HTML

from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json
import numpy as np

import pandas as pd
import map_methods
import math

In [35]:
infected_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=0&single=true&output=csv", 
                          names=['city', 'infected_in', 'date', 'count', 'age'], header=0)

infected_df

,city,infected_in,date,count,age
0,Скопје,Италија,2020-02-26,1,50-59
1,Дебар,Италија,2020-03-06,2,60-69
2,Дебар,Дебар,2020-03-09,2,60-69
3,Скопје,Италија,2020-03-09,1,50-59
4,Дебар,Дебар,2020-03-09,1,40-49
5,Дебар,Дебар,2020-03-11,2,60-69
6,Дебар,Дебар,2020-03-13,4,50-59
7,Скопје,Шпанија,3/14/2020,1,непознато
8,Дебар,Дебар,3/14/2020,5,непознато


In [36]:
healed_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=757629356&single=true&output=csv",
                       names=['city', 'date', 'count', 'age'], header=0)
healed_df

,city,date,count,age


In [37]:
dead_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vR-Ul4NMiPvca7QH-RlYk2Q1hrVmjjGp5tr5n64l1z-SH5S2NoMeqjSd5Ulo171tHKM2Crfr7u0tpcz/pub?gid=1754038661&single=true&output=csv",
                                               header=0, names=['city', 'date', 'count', 'age'])
dead_df

,city,date,count,age


In [83]:
infected_by_city = infected_df[['city', 'count']].copy().groupby('city').sum()
healded_by_city = healed_df[['city', 'count']].copy().groupby('city').sum()
dead_by_city = dead_df[['city', 'count']].copy().groupby('city').sum()
covid_mk = pd.merge(infected_by_city, healded_by_city, on='city', how='outer', suffixes=['_infected', '_healed'])
covid_mk['count'] = covid_mk.count_infected
covid_mk = pd.merge(covid_mk, dead_by_city, on='city', how='outer', suffixes=['', '_dead'])
covid_mk.fillna(int(0), inplace=True)
covid_mk['count'] = covid_mk.count_infected - covid_mk.count_healed - covid_mk.count_dead
covid_mk = covid_mk.astype(int)
covid_mk

,count_infected,count_healed,count,count_dead
city,,,,
Дебар,16,0,16,0
Скопје,3,0,3,0


In [184]:
world_cities = geopandas.read_file("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip")

mk_cities = world_cities[world_cities["iso_a2"] == 'MK']
mk_cities

,featurecla,scalerank,adm1_code,diss_me,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,...,name_nl,name_pl,name_pt,name_ru,name_sv,name_tr,name_vi,name_zh,ne_id,geometry
235,Admin-1 scale rank,10,MKD-4845,4845,MK-72,http://en.wikipedia.org/wiki/Municipality_of_S...,MK,1,Struga,None,...,Struga,Gmina Struga,Município de Struga,Струга,Struga,Struga Belediyesi,Đô thị tự trị Struga,斯特魯加區,1159315919,"POLYGON ((20.49604 41.33779, 20.51041 41.34440..."
237,Admin-1 scale rank,10,MKD-2951,2951,MK-78,None,MK,1,Centar župa,None,...,Centar Župa,Gmina Centar Żupa,Município Central de Zupa,Центар Жупа,Centar Župa,Merkez Jupa Belediyesi,Đô thị tự trị Centar Zupa,辛塔尔祖帕区,1159314519,"POLYGON ((20.53905 41.40294, 20.53407 41.41298..."
238,Admin-1 scale rank,10,MKD-2952,2952,MK-21,None,MK,1,Debar,None,...,Debar,Gmina Debar,Município de Debar,Дебар,Debar,Debre Belediyesi,Đô thị tự trị Debar,德巴尔区,1159314521,"POLYGON ((20.48160 41.46827, 20.47082 41.48377..."
239,Admin-1 scale rank,10,MKD-2955,2955,MK-50,None,MK,1,Mavrovo and Rostusa,None,...,Mavrovo en Rostuša,Gmina Mawrowo-Rostusza,Município de Mavroso e Rostusa,Маврово и Ростуша,Mavrovo i Rostusja,Mavrova ve Rostuşa Belediyesi,Đô thị tự trị Mavrovo và Rostusa,馬夫羅沃和羅斯圖沙區,1159314529,"POLYGON ((20.53467 41.58732, 20.53459 41.59403..."
240,Admin-1 scale rank,10,MKD-2962,2962,MK-19,None,MK,1,Gostivar,None,...,Gostivar Municipality,Gmina Gostivar,Município de Gostivar,Гостивар,Opsjtina Gostivar,Gostivar Belediyesi,Đô thị tự trị Gostivar,戈斯蒂瓦区,1159314545,"POLYGON ((20.55014 41.80006, 20.54844 41.81419..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,Admin-1 scale rank,10,MKD-3003,3003,MK-14,None,MK,1,Vinica,Vinitsa,...,Vinica,Gmina Winica,Município de Vinica,Виница,Vinica,Vinica Belediyesi,Đô thị tự trị Vinica,维尼察区,1159314567,"POLYGON ((22.44621 41.77016, 22.44662 41.77750..."
4256,Admin-1 scale rank,10,MKD-2997,2997,MK-07,None,MK,1,Bosilovo,None,...,Bosilovo,Gmina Bosiłowo,Município de Bosilovo,Босилово,Bosilovo,Bosilovo Belediyesi,Đô thị tự trị Bosilovo,博西洛沃区,1159314557,"POLYGON ((22.85807 41.51705, 22.85900 41.51230..."
4257,Admin-1 scale rank,10,MKD-2905,2905,MK-64,None,MK,1,Radoviš,Radovis,...,Radoviš,Gmina Radoviš,Município de Radovis,Радовиш,Radovisj,Radoviş Belediyesi,Đô thị tự trị Radovis,拉多維什區,1159314419,"POLYGON ((22.40104 41.71273, 22.40487 41.71906..."
4258,Admin-1 scale rank,10,MKD-2995,2995,MK-11,None,MK,1,Vasilevo,None,...,Vasilevo,Gmina Vasilevo,Vasilevo,Василево,Vasilevo,Vasilev Municipality,Đô thị tự trị Vasilevo,瓦西列沃區,1159314553,"POLYGON ((22.55411 41.48615, 22.54263 41.49090..."


In [86]:
mk_cities['city'] = mk_cities['name_ru']
covid_mk_cities = pd.merge(mk_cities, covid_mk, on='city', how='left')

/home/aleks/anaconda3/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [133]:
covid_mk_cities['count'] = covid_mk_cities['count'].fillna(0).astype(int)
covid_mk_cities['count_infected'] = covid_mk_cities['count_infected'].fillna(0).astype(int)
covid_mk_cities['count_healed'] = covid_mk_cities['count_healed'].fillna(0).astype(int)
covid_mk_cities['count_dead'] = covid_mk_cities['count_dead'].fillna(0).astype(int)

In [147]:
covid_active = dict(zip(map(str, covid_mk_cities.index), np.log2(covid_mk_cities['count'] + 1)))
covid_mk_cities_geo_json = json.loads(covid_mk_cities.to_json())
covid_mk_cities_geo_json
covid_active

{'0': 0.0,
 '1': 0.0,
 '2': 4.087462841250339,
 '3': 0.0,
 '4': 0.0,
 '5': 0.0,
 '6': 0.0,
 '7': 0.0,
 '8': 0.0,
 '9': 0.0,
 '10': 0.0,
 '11': 0.0,
 '12': 0.0,
 '13': 0.0,
 '14': 0.0,
 '15': 0.0,
 '16': 0.0,
 '17': 0.0,
 '18': 0.0,
 '19': 0.0,
 '20': 0.0,
 '21': 0.0,
 '22': 0.0,
 '23': 0.0,
 '24': 0.0,
 '25': 0.0,
 '26': 0.0,
 '27': 0.0,
 '28': 0.0,
 '29': 0.0,
 '30': 0.0,
 '31': 0.0,
 '32': 0.0,
 '33': 0.0,
 '34': 0.0,
 '35': 0.0,
 '36': 0.0,
 '37': 0.0,
 '38': 0.0,
 '39': 0.0,
 '40': 0.0,
 '41': 0.0,
 '42': 0.0,
 '43': 0.0,
 '44': 0.0,
 '45': 0.0,
 '46': 0.0,
 '47': 0.0,
 '48': 0.0,
 '49': 0.0,
 '50': 0.0,
 '51': 0.0,
 '52': 0.0,
 '53': 0.0,
 '54': 0.0,
 '55': 0.0,
 '56': 0.0,
 '57': 0.0,
 '58': 0.0,
 '59': 0.0,
 '60': 0.0,
 '61': 0.0,
 '62': 0.0,
 '63': 0.0,
 '64': 0.0,
 '65': 0.0,
 '66': 0.0,
 '67': 0.0,
 '68': 0.0,
 '69': 0.0,
 '70': 0.0,
 '71': 0.0,
 '72': 0.0,
 '73': 0.0,
 '74': 0.0,
 '75': 0.0,
 '76': 0.0,
 '77': 0.0,
 '78': 0.0,
 '79': 0.0,
 '80': 0.0,
 '81': 0.0,
 '82': 0.0,


In [211]:
import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
    
layer = ipyleaflet.Choropleth(
    geo_data=covid_mk_cities_geo_json,
    choro_data=covid_active,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.3, 'weight': 0.05}
)

geo_data = GeoData(
    geo_dataframe = covid_mk_cities,
    style={'color': 'black', 'fillColor': 'yellow', 'opacity':0.00, 'weight':0, 'dashArray':'2', 'fillOpacity':0.0},
    hover_style={'fillColor': 'grey' , 'fillOpacity': 0.3},
    name = 'Countries'
)

m = map_methods.get_macedonia_map()
m.add_layer(layer)
m.add_layer(geo_data)
m.add_control(LayersControl())
m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [39]:
merged = pd.merge(merged, city_df, on='city', how='inner')
merged = merged.set_index("city")
merged

,count_x,count_y,count,id,city_cyrilic,admin_name_cyrilic,city_latin,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
city,,,,,,,,,,,,,,,,


In [19]:


m = map_methods.get_macedonia_map()

def hide_marker(name, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    return (infected - healed - dead) < 1
    

def get_marker(name, location, row):
    infected = row['count_x']
    healed = row['count_y']
    dead = row['count']
    current = infected - healed - dead
    scale = math.log(current, 2)
    return CircleMarker(location=location, radius=(int)(5*scale), color='red', weight=3)

map_methods.add_marker_layer(m=m, d=merged, 
                             cols=["count_x", "count_y", "count", "population"], 
                             names=["Заразени", "Излечени" ,"Мртви", "Население"],
                            get_marker=get_marker, hide_marker=hide_marker)

m

Map(center=[41.6086, 21.7453], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

NameError: name 'hospital_data' is not defined